# Trusted Context Lab

Prepare for lab:

In [34]:
%run Lab_macros.ipynb

DB2 Extensions Loaded.


alias 'run-as' defined


Execute the following commands as instance owner db2inst1 in a terminal window.
But first you have to replace XXX.XXX.XXX.XXX with the IPV4 address of your system. (Use ifconfig in the terminal window to get the IP address. Don't select the IP of the local loopback device, use your regular Ethernet interface. The IPV4 address is prefixed by "inet".)

db2 catalog tcpip node n1 remote XXX.XXX.XXX.XXX server 50000

db2 catalog database sample as samplecl at node n1

db2 terminate

The reason why we do this is that trusted contexts can only be used with a TCP/IP connection from a remote system (client or other server). To avoid a second virtual machine we simulate this by connection via the database sample1 which we just cataloged.

Now let us create tables with two different users (peter and paul) and insert some rows. These users themselves have already been created in the previous lab.

In [35]:
%sql connect to sample user peter using p123

Connection successful.


In [36]:
%%sql
CREATE TABLE t1 (c0 INT, c1 CHAR(10));

Command completed.


In [37]:
%%sql
INSERT INTO t1 VALUES (0,'peter');
INSERT INTO t1 VALUES (1,'peter');

Command completed.


In [38]:
%sql connect to sample user paul using p123

Connection successful.


In [39]:
%%sql
CREATE TABLE t2 (c0 INT, c1 CHAR(10));

Command completed.


In [40]:
%%sql
INSERT INTO t2 VALUES (0,'paul');
INSERT INTO t2 VALUES (1,'paul');

Command completed.


# Reading from the tables

Let's check who can access which table

In [41]:
%sql connect to sample user peter using p123

Connection successful.


In [42]:
%%sql
select * from peter.t1;

,C0,C1
0,0,peter
1,1,peter


In [43]:
%%sql
select * from paul.t2;

Command completed.


In [44]:
%sql connect to sample user paul using p123

Connection successful.


In [45]:
%%sql
select * from peter.t1;

Command completed.


In [46]:
%%sql
select * from paul.t2;

,C0,C1
0,0,paul
1,1,paul


# We expect to see the same when connecting via sample1

Let's check it.

In [47]:
%sql connect to samplecl user peter using p123

Connection successful.


In [48]:
%%sql
select * from peter.t1;

,C0,C1
0,0,peter
1,1,peter


In [49]:
%%sql
select * from paul.t2;

Command completed.


In [50]:
%sql connect to samplecl user paul using p123

Connection successful.


In [51]:
%%sql
select * from peter.t1;

Command completed.


In [52]:
%%sql
select * from paul.t2;

,C0,C1
0,0,paul
1,1,paul


# Use of trusted context to give paul access to peter's table in a specific context

The user creating the Trusted Context has to have SECADM the privilege.

%sql connect to sample user db2inst1 using db2inst1

We will set a default role for user paul in the trusted context. This role should give paul access to peter's table.

First we create the role:

In [54]:
%%sql
create role role_peter;

Command completed.


Next we grant SELECT permission on peter's table to this role:

In [55]:
%sql grant select on peter.t1 to role_peter

Command completed.


Now we are ready to define the trusted context.
    
We want to allow give user paul the permissions associated with the role role_peter when paul is connecting from a specific system to the Db2 server. For the purposes of this lab this system will be just our virtual machine. Therefore, we have to get our IPV4 address (how to get this address is described at the beginning of this notebook.) and replace the XXX.XXX.XXX.XXX in the cell below by this IP address. 

In [57]:
%%sql
create trusted context tc based upon connection 
using system authid paul 
attributes (address '192.168.251.133') 
default role role_peter enable;

Command completed.


Now we are ready to test the trusted context. For this we connect as user paul to the database samplecl (which is our remote client for the purposes of this lab)

In [58]:
%sql connect to samplecl user paul using p123

Connection successful.


In [59]:
%%sql
select * from peter.t1;

,C0,C1
0,0,peter
1,1,peter


We were now able to read peter.t1 because of the default role specified in the trusted context.

In [60]:
%sql connect to sample user paul using p123

Connection successful.


In [61]:
%%sql
select * from peter.t1;

Command completed.


When connecting locally to the database on which the trusted context was defined, we don't have the permission to access the table.

# Disabling and Enabling Trusted Contexts

We had created our trusted context already in the enabled state. But trusted contexts can be enabled and disabled. Let's check it:

In [68]:
%sql connect to sample user db2inst1 using db2inst1

Connection successful.


In [70]:
%sql alter trusted context tc alter disable

Command completed.


In [71]:
%sql connect to sample1 user paul using p123

Connection successful.


In [72]:
%%sql
select * from peter.t1;

Command completed.


In [73]:
%sql connect to sample user db2inst1 using db2inst1

Connection successful.


In [74]:
%sql alter trusted context tc alter enable

Command completed.


In [75]:
%sql connect to sample1 user paul using p123

Connection successful.


In [76]:
%%sql
select * from peter.t1;

,C0,C1
0,0,peter
1,1,peter


# How is a trusted context stored?

In [63]:
%sql connect to sample user db2inst1 using db2inst1

Connection successful.


In [64]:
%sql connect to sample 

Connection successful.


In [65]:
%%sql
select * from syscat.contexts;

,CONTEXTNAME,CONTEXTID,SYSTEMAUTHID,DEFAULTCONTEXTROLE,CREATE_TIME,ALTER_TIME,ENABLED,AUDITPOLICYID,AUDITPOLICYNAME,AUDITEXCEPTIONENABLED,REMARKS
0,SYSATSCONTEXT,100,SYSATS,None,2019-05-23 11:10:40.008859,2019-05-23 11:10:40.008859,Y,None,None,N,None
1,TC,102,PAUL,ROLE_PETER,2019-09-19 03:18:06.480854,2019-09-19 03:18:06.480854,Y,None,None,N,None


The entry with contextname "TC" corresponds to the trusted context we created. Which of the information did we specify explicitly, which was created implicitly?

In [66]:
%%sql
select * from syscat.contextattributes;

,CONTEXTNAME,ATTR_NAME,ATTR_VALUE,ATTR_OPTIONS
0,SYSATSCONTEXT,ENCRYPTION,NONE,None
1,TC,ENCRYPTION,NONE,None
2,TC,ADDRESS,192.168.251.133,None


Why are there two rows for our trusted context in this table?

In [67]:
%%sql
select * from syscat.surrogateauthids;

,GRANTOR,TRUSTEDID,TRUSTEDIDTYPE,SURROGATEAUTHID,SURROGATEAUTHIDTYPE,AUTHENTICATE,CONTEXTROLE,GRANT_TIME
0,DB2INST1,SYSATSCONTEXT,C,PUBLIC,G,N,None,2019-05-23 11:10:40.008859


There are no entries for the trusted context we created in the surrogateauthids table. We would have had specify users in the trusted context to which we might want to switch. Check the documentation (in IBM Knowledge Center) for trusted context on this and think how a example using this functionality could look like.

You are done with the trusted context lab now.

# Clean up

If you have messed up this trusted context lab, you can clean up all objects in the database and start from the top of this notebook.

In [28]:
%sql connect to sample user db2inst1 using db2inst1

Connection successful.


In [29]:
%sql drop table peter.t1

Command completed.


In [30]:
%sql drop table paul.t2

Command completed.


In [32]:
%sql drop trusted context tc

Command completed.


In [33]:
%sql drop role role_peter

Command completed.
